In [50]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt

img = cv.imread('5_of_diamonds.png')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

# Divide images into two regions, blacks and whites
ret, thresh = cv.threshold(gray, 0, 255, cv.THRESH_BINARY_INV | cv.THRESH_OTSU)

# Remove noise (big white regions swallow little black regions)
kernel = np.ones((3,3), np.uint8)
opening = cv.morphologyEx(thresh, cv.MORPH_OPEN, kernel, iterations = 2)

# Find the sure background region
sure_bg = cv.dilate(opening, kernel, iterations = 3)

# Find the sure foreground region
dist_transform = cv.distanceTransform(opening, cv.DIST_L2, 5)
ret, sure_fg = cv.threshold(dist_transform, 0.7*dist_transform.max(), 255, 0)
sure_fg = sure_fg.astype(np.uint8)

# Find the unknown region
unknown = cv.subtract(sure_bg, sure_fg)

# Label the foreground objects
ret, markers = cv.connectedComponents(sure_fg)

# Add one to all labels so that sure background is not 0, but 1
markers += 1

# Label the unknown region as 0
markers[unknown == 255] = 0

# Label -1 to edges and blue line on contours
gray_converted_to_colour = cv.cvtColor(gray, cv.COLOR_GRAY2RGB)
markers = cv.watershed(gray_converted_to_colour, markers)
gray_converted_to_colour[markers == -1] = [255, 0, 0]
cv.imwrite('final.png', gray_converted_to_colour)

True